In [1]:
library(fit)
library(tidyverse)
#gpxsee to inspect fit files

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()



# Decoder script
## Helper functions

In [2]:
#convert timestamp to date (weird, but off by only some minutes)
to_date <- function (t) {as.POSIXct(as.numeric(as.character(t)), origin="1990-01-01", tz="GMT+23")}
#array: hours k[1], mins k[2], secs k[3] (from int (seconds))
to_duration <- function (t,x="h") {
            if(x!="s"){
                t <- round(t/1000) #normalize if t comes from f.ex total_elapsed_time
            }
            hours <- t%/%3600
            mins <- t%%3600%/%60
            secs <- t%%60
            paste(hours,":",floor(mins),":",floor(secs))
             }
#km/h -> min/km, mins k[1], secs k[2]
to_pace <- function (t){
    h <- 3600/t
    hours <- h%/%3600
    mins <- h%%3600%/%60
    secs <- h%%60
    #ifelse(secs >= 60, paste(mins+1,":",secs-60,sep=""),paste(mins,":",secs,sep=""))
    paste0(hours,":",floor(mins),":",floor(secs))
}
#m/s -> km/h
to_kmh <- function (ms) {ms*3.6}
#m/s -> min/100m 
to_swimpace <- function (ms) {10**4/(ms*6)}

In [3]:
to_pace(11.5)


[1] "0:5:13"

In [4]:
#input: int from FIT, out: string
get_sport <- function(f){
    case_when(
        f == 2 ~ "Bike",
        f == 17 ~ "Hiking",
        f == 1 ~ "Running",
        f == 5 ~ "Swimming",
        TRUE ~ "UNKNOWN_SPORT"
    )
}

In [5]:
get_running_session <- function(buf){
    data.frame("avg_hr"            = buf$avg_heart_rate, 
               "max_hr"            = buf$max_heart_rate,
               "avg_speed"         = round(to_kmh(buf$avg_speed/1000),1),
               "max_speed"         = round(to_kmh(buf$max_speed/1000),1),
               "avg_pace"          = to_pace(round(to_kmh(buf$avg_speed/1000),1)),
               "max_pace"          = to_pace(round(to_kmh(buf$max_speed/1000),1)),
               "total_ascent"      = buf$total_ascent, # TODO 
               "total_descent"     = buf$total_descent, # TODO
               "total_distance"    = round(buf$total_distance/10**5,2),
               "total_time"        = to_duration(buf$total_elapsed_time),
               "total_time_moving" = to_duration(buf$total_timer_time),
               "num_laps"          = buf$num_laps,
               "training_effect"   = buf$total_training_effect/10
              )
}
get_running_lap <- function(buf){
    data.frame("avg_hr"            = buf$avg_heart_rate,
               "avg_speed"         = round(to_kmh(buf$avg_speed/1000),1),
               "max_speed"         = round(to_kmh(buf$max_speed/1000),1),
               "avg_pace"          = to_pace(round(to_kmh(buf$avg_speed/1000),1)),
               #"time"            = to_date(buf$timestamp),
               "total_distance"    = round(buf$total_distance/10**5,2),
               "total_time"        = to_duration(buf$total_elapsed_time),
               "total_time_moving" = to_duration(buf$total_timer_time)
              )
}

## Start decoding here!

Plan: Read all files in ACTIVITY directory, and convert all *.FIT files to *.JSON files.

The *.FIT files are deleted then

In [6]:
act_files <- list.files('f/')
#contains all fit files to be processed
fit_files <- c()
for (f in act_files){
    sp <- unlist(strsplit(f,"[.]"))
    #print(sp[1:length(sp)-1])
    if(sp[length(sp)] == "FIT" | sp[length(sp)] == "fit"){
        #file is a fit file! save its name
        fit_files <- c(fit_files,sp[1:length(sp)-1])
    }
}

In [7]:
file <- read.fit(paste('f/',fit_files[1],'.FIT',sep=""))

ERROR: Error in decode_fit_file(filename): File not found 


In [ ]:
file_sport_type <- get_sport(file$sport$sport)
file_date <- to_date(file$session$start_time)

#for running
file_session <- get_running_session(file$session[,c('avg_heart_rate','max_heart_rate',
                                                    'avg_speed','max_speed',
                                                    'total_ascent','total_descent', 'total_distance',
                                                    'total_elapsed_time', 'total_timer_time',
                                                    'num_laps', 'total_training_effect')])
file_lap <- get_running_lap(file$lap[,c('avg_heart_rate','avg_speed','max_speed','timestamp',
                                        'total_distance','total_elapsed_time','total_timer_time')])
print(file$record[1,])
buf <- file$record[,c('distance','heart_rate','speed','position_lat','position_long','timestamp')]
head(buf)
data.frame("distance" = round(buf$distance/10**5,3),
           "hr" = buf$heart_rate,
           "speed" = round(to_kmh(buf$speed/1000),1),
           "pace" = to_pace(round(to_kmh(buf$speed/1000),1)),
           "pos_lat" = buf$position_lat,
           "pos_long" = buf$position_long,
           "time" = to_duration(buf$timestamp-file$session$start_time,"s"))

# This part was used to analyze the data decoded by the lib

In [10]:
#rad
data <- read.fit('data/ACTIVITY/A8FF5035.FIT')
#wandern
data2 <- read.fit('ACTIVITY/A8691324.FIT')
#joggen
data3 <- read.fit('ACTIVITY/A6BG4226.FIT')
#schwimmen
data4 <- read.fit('ACTIVITY/A7CG0124.FIT')

ERROR: Error in decode_fit_file(filename): File not found 


In [11]:
data

function (..., list = character(), package = NULL, lib.loc = NULL, 
    verbose = getOption("verbose"), envir = .GlobalEnv, overwrite = TRUE) 
{
    fileExt <- function(x) {
        db <- grepl("\\.[^.]+\\.(gz|bz2|xz)$", x)
        ans <- sub(".*\\.", "", x)
        ans[db] <- sub(".*\\.([^.]+\\.)(gz|bz2|xz)$", "\\1\\2", 
            x[db])
        ans
    }
    my_read_table <- function(...) {
        lcc <- Sys.getlocale("LC_COLLATE")
        on.exit(Sys.setlocale("LC_COLLATE", lcc))
        Sys.setlocale("LC_COLLATE", "C")
        read.table(...)
    }
    names <- c(as.character(substitute(list(...))[-1L]), list)
    if (!is.null(package)) {
        if (!is.character(package)) 
            stop("'package' must be a character string or NULL")
        if (FALSE) {
            if (any(package %in% "base")) 
                warning("datasets have been moved from package 'base' to package 'datasets'")
            if (any(package %in% "stats")) 
                warning("datasets have been moved from package 'stats' to package 'datasets'")
            package[package %in% c("base", "stats")] <- "datasets"
        }
    }
    paths <- find.package(package, lib.loc, verbose = verbose)
    if (is.null(lib.loc)) 
        paths <- c(path.package(package, TRUE), if (!length(package)) getwd(), 
            paths)
    paths <- unique(normalizePath(paths[file.exists(paths)]))
    paths <- paths[dir.exists(file.path(paths, "data"))]
    dataExts <- tools:::.make_file_exts("data")
    if (length(names) == 0L) {
        db <- matrix(character(), nrow = 0L, ncol = 4L)
        for (path in paths) {
            entries <- NULL
            packageName <- if (file_test("-f", file.path(path, 
                "DESCRIPTION"))) 
                basename(path)
            else "."
            if (file_test("-f", INDEX <- file.path(path, "Meta", 
                "data.rds"))) {
                entries <- readRDS(INDEX)
            }
            else {
                dataDir <- file.path(path, "data")
                entries <- tools::list_files_with_type(dataDir, 
                  "data")
                if (length(entries)) {
                  entries <- unique(tools::file_path_sans_ext(basename(entries)))
                  entries <- cbind(entries, "")
                }
            }
            if (NROW(entries)) {
                if (is.matrix(entries) && ncol(entries) == 2L) 
                  db <- rbind(db, cbind(packageName, dirname(path), 
                    entries))
                else warning(gettextf("data index for package %s is invalid and will be ignored", 
                  sQuote(packageName)), domain = NA, call. = FALSE)
            }
        }
        colnames(db) <- c("Package", "LibPath", "Item", "Title")
        footer <- if (missing(package)) 
            paste0("Use ", sQuote(paste("data(package =", ".packages(all.available = TRUE))")), 
                "\n", "to list the data sets in all *available* packages.")
        else NULL
        y <- list(title = "Data sets", header = NULL, results = db, 
            footer = footer)
        class(y) <- "packageIQR"
        return(y)
    }
    paths <- file.path(paths, "data")
    for (name in names) {
        found <- FALSE
        for (p in paths) {
            tmp_env <- if (overwrite) 
                envir
            else new.env()
            if (file_test("-f", file.path(p, "Rdata.rds"))) {
                rds <- readRDS(file.path(p, "Rdata.rds"))
                if (name %in% names(rds)) {
                  found <- TRUE
                  if (verbose) 
                    message(sprintf("name=%s:\t found in Rdata.rds", 
                      name), domain = NA)
                  thispkg <- sub(".*/([^/]*)/data$", "\\1", p)
                  thispkg <- sub("_.*$", "", thispkg)
                  thispkg <- paste0("package:", thispkg)
                  objs <- rds[[name]]
                  lazyLoad(file.path(p, "Rdata"), envir = tmp_env, 
                    filter = functio